# Load → Split → Embed 전체 파이프라인

이 노트북에서는 **문서 로드 → 분할 → 임베딩**까지의 전체 흐름을 연결하여 실행합니다.

## RAG 전처리 파이프라인

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  📄 Load    │ ──▶ │  ✂️ Split   │ ──▶ │  🔢 Embed   │ ──▶ │  💾 Store   │
│ TextLoader  │     │ TextSplitter│     │  Embeddings │     │ VectorStore │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
    ↑                    ↑                    ↑
    └────────────────────┴────────────────────┘
              이 노트북에서 다루는 범위
```

---

# 1. Ollama 설치 및 서버 실행

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

# 2. 패키지 설치 및 모델 다운로드

In [ ]:
!ollama pull nomic-embed-text
!pip install -q langchain langchain-community langchain-ollama langchain-text-splitters

# 3. 테스트 파일 생성

In [ ]:
sample_text = '''LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다.

주요 기능:
1. 프롬프트 관리 및 최적화
2. 체인 구성을 통한 복잡한 워크플로우 생성
3. 다양한 데이터 소스와의 연동
4. 메모리 관리를 통한 대화 컨텍스트 유지

LangChain을 사용하면 RAG(Retrieval-Augmented Generation) 시스템을 쉽게 구축할 수 있습니다. RAG는 외부 지식을 검색하여 LLM의 응답을 향상시키는 기술입니다.

Vector Store는 임베딩된 문서를 저장하고 유사도 검색을 수행하는 데이터베이스입니다. 대표적으로 Chroma, Pinecone, PGVector 등이 있습니다.

임베딩은 텍스트를 고차원 벡터로 변환하는 과정입니다. 의미가 비슷한 텍스트는 벡터 공간에서 가까운 위치에 있습니다.
'''

with open('./test.txt', 'w', encoding='utf-8') as f:
    f.write(sample_text)

print(f"test.txt 생성 완료 ({len(sample_text)}자)")

# 4. 전체 파이프라인 실행

## Step 1: 문서 로드

In [ ]:
from langchain_community.document_loaders import TextLoader

# 문서 로드
loader = TextLoader('./test.txt', encoding='utf-8')
doc = loader.load()

print(f"✅ Step 1: 문서 로드 완료")
print(f"   - 로드된 문서 수: {len(doc)}")
print(f"   - 원본 길이: {len(doc[0].page_content)}자")

## Step 2: 문서 분할

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 분할
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)
chunks = splitter.split_documents(doc)

print(f"\n✅ Step 2: 문서 분할 완료")
print(f"   - 분할된 청크 수: {len(chunks)}")
print(f"   - 청크 크기: {[len(c.page_content) for c in chunks]}")

## Step 3: 임베딩 생성

In [ ]:
from langchain_ollama import OllamaEmbeddings

# 임베딩 모델 생성
embeddings_model = OllamaEmbeddings(model='nomic-embed-text')

# 각 청크의 텍스트를 임베딩
embeddings = embeddings_model.embed_documents(
    [chunk.page_content for chunk in chunks]
)

print(f"\n✅ Step 3: 임베딩 생성 완료")
print(f"   - 임베딩 수: {len(embeddings)}")
print(f"   - 벡터 차원: {len(embeddings[0])}")

# 5. 결과 확인

In [ ]:
# 청크와 임베딩 매핑 확인
print("=== 청크-임베딩 매핑 ===")
for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
    print(f"\n[청크 {i+1}]")
    print(f"  텍스트: {chunk.page_content[:50]}...")
    print(f"  벡터: [{emb[0]:.4f}, {emb[1]:.4f}, ..., {emb[-1]:.4f}]")

---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')

# 변경 (Ollama)
from langchain_ollama import OllamaEmbeddings
embeddings_model = OllamaEmbeddings(model='nomic-embed-text')
```

## 전체 파이프라인 요약

```python
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

# 1. Load
docs = TextLoader('./file.txt').load()

# 2. Split
chunks = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

# 3. Embed
embeddings = OllamaEmbeddings(model='nomic-embed-text').embed_documents(
    [c.page_content for c in chunks]
)
```

## 다음 단계

생성된 임베딩을 **Vector Store**에 저장하여 유사도 검색 가능하게 만듭니다. (09번 노트북)